# Masterclass

### Importieren Bibliotheken

An dieser Stelle werden die Bibliotheken importiert, die für die Datenanalyse notwendig sind.

In [16]:
import ROOT
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import math

### Erstellen DataFrame

Hier wird ein Pandas Dataframe erstellt. Darin werden die Daten gespeichert, die für die Analyse von Nöten sind. Wir nutzen sie, um die Histogramme zu füllen.
Die Datei wird zusätzlich noch ausgelesen, um eine zweite Art von Histogramm zu erstellen, aus denen bestimmte Punkte besser abzulesen sind!

In [8]:
#2015MU in Panda DF
df = uproot.open("/eos/user/s/sharst/SWAN_projects/MasterclassTest/Data2015.root")["DecayTree"].arrays(library="pd")
gf = uproot.open("/eos/user/s/sharst/SWAN_projects/MasterclassTest/Data2015WS.root")["DecayTree"].arrays(library="pd")

f = ROOT.TFile.Open("/eos/user/s/sharst/SWAN_projects/MasterclassTest/Data2015.root")
tree1 = f.Get("DecayTree")

g = ROOT.TFile.Open("/eos/user/s/sharst/SWAN_projects/MasterclassTest/Data2015WS.root")
tree2 = g.Get("DecayTree")

### Plots Xi_c
#### Anpassen Cuts mit Slider
In der nächsten Zelle sollst Du zum ersten Mal selsbt arbeiten, hier beginnt die eigentliche Datenanalyse! Du sollst ein sauberes Signal von Xi_c erhalten, was möglichst wenig Hintergrund hat. Dafür wendest Du Cuts auf die Daten an, um diese zuzuschneiden.

Das Signal für Xi_c soll im orangefarbenen Histogramm bestimmt werden. In den blauen Histogrammen kannst Du sehen, wie die Werteverteilung der Cuts ist. Daran kannst Du erahnen, welche Cuts sinnvoll sind und welche nicht.

Es kann zunächst einen kurzen Moment dauern, bis die Histogramme erstellt sind, aber dann kann es direkt losgehen. Mithilfe der Schieberegler können die einzelnen Cuts angepasst und so die Daten zugeschnitten werden. Eine Beschreibung, was welcher Cut bedeutet, findest du auf dem Aufgabenblatt.

Wenn Du einen Schieberegler verstellt hast, wartest Du am besten kurz (5-10s), bis der zuletzt angewendete Cut von den Histogrammen übernommen wurde.

In [4]:
%matplotlib widget

fig = plt.figure()
fig.set_size_inches(10, 5)
fig.canvas.header_visible = False

ax = plt.subplot2grid((7,16),(1,1),colspan = 7, rowspan = 5)
bx = plt.subplot2grid((7,16),(1,9),colspan = 3, rowspan = 2)
dx = plt.subplot2grid((7,16),(4,9),colspan = 3, rowspan = 2)
cx = plt.subplot2grid((7,16),(1,13),colspan = 3, rowspan = 2)
ex = plt.subplot2grid((7,16),(4,13),colspan = 3, rowspan = 2)


plt.tight_layout()

a = 0

#fig, ((ax,bx,cx),(dx,ex)) = plt.subplots(2,3)
#fig.set_size_inches(16, 7)
#print("Filter 2: Wahrscheinlichkeit, dass ein Teilchen ein Proton ist")
#print("Filter 3: Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt")
#print("Filter 4: Qualität der Flugdistanz bezüglich des Kollisionspunkts")
print("Eine kurze Erinnerung: Ist der Slider auf der linken Seite, wird wenig eingeschränkt, rechts schränkt er stark ein.")
lb1 = widgets.Label("Filter 1: $P(e^-)$ Wahrscheinlichkeit, dass ein Teilchen ein Elektron ist")
lb2 = widgets.Label("Filter 2: $P(e^-)$ Wahrscheinlichkeit, dass ein Teilchen ein Proton ist")
lb3 = widgets.Label("Filter 3: $\chi^2_{Stoß}$ Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt")
lb4 = widgets.Label("Filter 4: $\chi^2_{Flug}$ Qualität der Flugdistanz bezüglich des Kollisionspunkts")
display(lb1)
display(lb2)
display(lb3)
display(lb4)

@widgets.interact(Cut1=widgets.FloatSlider(min = 0, max = 1, step = 0.05, description = "Ftr.1 $P(e^-)$",continuous_update = False), Cut2=widgets.FloatSlider(min = 0, max = 1, step = 0.05, description = "Ftr.2 $P(p)$",continuous_update = False), Cut3=widgets.FloatSlider(value=0,min = 0, max = 11, step = 0.1, description = "Ftr.3 $\chi^2_{Stoß}$",continuous_update = False), Cut4=widgets.IntSlider(min = 0, max = 500, step = 2, description = "Ftr.4 $\chi^2_{Flug}$",continuous_update = False))
def update(Cut1, Cut2=0, Cut3=11, Cut4=0):
    
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    
    ax.set_title("Kombination $p$ mit $K^-$ und $\pi^+$")
    ax.set_ylabel("Einträge pro 1.1MeV")
    ax.set_xlabel("$m(pK^-\pi^+)$ [$MeV$]")
    
    bx.set_title("")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Filter 1: Elektron")
    bx.set_yscale("log")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Filter 2: Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("Filter 3: Stoßparameter")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("Filter 4: Flugdistanz")
    
    ax.hist(df["lab1_M"], bins=100, range =(2420,2510), color = "brown")
    ax.hist(df["lab1_M"][df["lab2_ProbNNe"]>Cut1][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(2420,2510), color = "darkorange")
    bx.hist(df["lab2_ProbNNe"][df["lab2_ProbNNe"]>Cut1], bins=100, range =(0,1))
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Eine kurze Erinnerung: Ist der Slider auf der linken Seite, wird wenig eingeschränkt, rechts schränkt er stark ein.


Label(value='Filter 1: $P(e^-)$ Wahrscheinlichkeit, dass ein Teilchen ein Elektron ist')

Label(value='Filter 2: $P(e^-)$ Wahrscheinlichkeit, dass ein Teilchen ein Proton ist')

Label(value='Filter 3: $\\chi^2_{Stoß}$ Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt')

Label(value='Filter 4: $\\chi^2_{Flug}$ Qualität der Flugdistanz bezüglich des Kollisionspunkts')

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Ftr.1 $P(e^-)$', max=1.0, s…

### Plot Xi_c
#### Histogramm in ROOT
Die nächste Zelle fordert Dich auf, die Werte für die Cuts anzugeben, die Du in dem vorherigen Teil mit den Schiebereglern erörtert hast.

Als nächstes wird dann ein anderes Histogramm erzeugt, was nur das Spektrum von Xi_c zeigt. In diesem kannst Du Dir mit dem Mauszeiger Werte anzeigen lassen, indem Du den Mauszeiger an die Stelle in das Histogramm bewegst, von der Du den Wert wissen möchtest. Zusätzlich wird dir ein Bereich an der X-Achse angezeigt, sodass Du besser ablesen kannst.

In [ ]:
cut11 = ROOT.TCut("lab2_ProbNNe>###") #Einfügen Filter1
cut12 = ROOT.TCut("lab2_ProbNNp>###") #Einfügen Filter2
cut13 = ROOT.TCut("lab1_IPCHI2_OWNPV<###") #Einfügen Filter3
cut14 = ROOT.TCut("lab1_FDCHI2_OWNPV>###") #Einfügen Filter4
cut10 = ROOT.TCut(cut11+cut12+cut13+cut14)

In [13]:
cut11 = ROOT.TCut("lab2_ProbNNe>0.0") #Einfügen Filter1
cut12 = ROOT.TCut("lab2_ProbNNp>0.45") #Einfügen Filter2
cut13 = ROOT.TCut("lab1_IPCHI2_OWNPV<3") #Einfügen Filter3
cut14 = ROOT.TCut("lab1_FDCHI2_OWNPV>100") #Einfügen Filter4
cut10 = ROOT.TCut(cut11+cut12+cut13+cut14)

In [19]:
xmin = 2420
xmax = 2510
ROOT.gStyle.SetOptStat(0) #Entfernen HistLegende

#Erstellen Fitfunktion: linBG + Gauss + Gauss
f1 = ROOT.TF1('f1','[0] + x*[1] + [3] * exp(-(x - [4])^2/(2*[2]*[2]))+ [5] * exp(-(x - [7])^2/(2*[6]*[6]))', xmin, xmax) #Fitfunktion
f1.SetParameter(4, 2468) #Parameter für Gauss Tendenz setzen Mean
f1.SetParameter(3,1) #Parameter für Gauss Tendenz setzen existenz
f1.SetParameter(5,1) #existenz Gauss2
f1.SetParameter(7, 2468) #Mean Gauss2

#Erstellen und Füllen der Histogramme
h1 = ROOT.TH1F("h1", "Kombination der Teilchen", 100, xmin, xmax) #Hist mit Cut
h2 = ROOT.TH1F("h2", "Kombination der Teilchen", 100, xmin, xmax) #Hist ohn Cut
tree1.Draw("lab1_M>>h1",cut10) #Füllen Hist mit Cut
tree1.Draw("lab1_M>>h2") #Füllen Hist ohne Cut
c1 = ROOT.TCanvas("myCanvasName","Titel",800,600) #Erstellen Canvas
h2.SetLineColor(ROOT.kRed)
h2.Draw("MIN0") #Hist oC zeichnen (y-Achse ab 0)
h1.Draw("same") #Hist mC zeichnen in gleiches TCanvas

#erste Fits, Fitparameter für Integrale nehmen
f1.SetLineColor(ROOT.kBlue-6)
h1.Fit("f1","RQ") #Hist mC fitten
p10 = f1.GetParameter("p0") #getParForBG#Y-Intersect
p11 = f1.GetParameter("p1") #GetParForBG#slope
p22 = f1.GetParameter("p2") #GetParForSignal #sigma
p23 = f1.GetParameter("p3") #Signal
p24 = f1.GetParameter("p4") #Signal #Mean
p25 = f1.GetParameter("p5") #Signal 
p26 = f1.GetParameter("p6") #Signal #sigma
p27 = f1.GetParameter("p7") #Signal #mean

#print("Steigung Background mit Cut: ", p11) #SlopeOfBGCUt
#print("Achsenabschnitt Background mit Cut: ", p10) #Y-IntersectOfBGCut
#print(" ") #Space

#Bestimmen der Funktione für Integrale, kosmetische Anpassungen Fits
f21 = ROOT.TF1("f21", "{p10}+x*{p11}".format(p10=p10, p11=p11), 0, xmax) #BGFunctionCut
f21.SetLineColor(1) #BGCut
f21.SetLineStyle(4) #BGCut
f21.Draw("same") #Draw BGCut
f22 = ROOT.TF1("f22", "{p23} * exp(-(x - {p24})^2/(2*{p22}*{p22}))+ {p25} * exp(-(x - {p27})^2/(2*{p26}*{p26}))".format(p22=p22,p23=p23,p24=p24,p25=p25,p26=p26,p27=p27), xmin,xmax) #only SignalPeak Fit
f23 = ROOT.TF1("f23", "{p10}+x*{p11} +{p23} * exp(-(x - {p24})^2/(2*{p22}*{p22}))+ {p25} * exp(-(x - {p27})^2/(2*{p26}*{p26}))".format(p10=p10,p11=p11,p22=p22,p23=p23,p24=p24,p25=p25,p26=p26,p27=p27), xmin,xmax)
f22.SetLineColor(ROOT.kBlue+2)
f22.SetLineWidth(3)
f22.Draw("same") #Draw SignalPeak
SB = f23.Integral(p24 - 2 * abs(p22), p24 + 2 * abs(p22)) #integrate Sign+Bg in signalregion (mean+-2sigma)
Signal = f22.Integral(p24 - 2*abs(p22), p24 + 2*abs(p22)) #integrate signal in signalregion ("")
f23.SetLineColor(1)
f23.SetLineStyle(4)
f23.Draw("same")
#Bg = f21.Integral(xmin, xmax) #Integrate Background whole Dataset

#Variablen für die
Varianz = abs(p22)
print("Signal:", Signal)
print("Signal + Untergrund:", SB)
Purity = math.sqrt(Signal/SB)
print("Reinheit:", Purity)
print("Breite:", Varianz)
print(" ")

f1.SetParameter(4, 2468) #Parameter für Gauss Tendenz setzen Mean
f1.SetParameter(3,1) #Parameter für Gauss Tendenz setzen existenz
f1.SetParameter(5,1) #existenz Gauss2
f1.SetParameter(7, 2468) #Mean Gauss2
f1.SetLineColor(ROOT.kRed)
h2.Fit("f1","RQ") #ursprünglicher Signal
p0 = f1.GetParameter("p0")
p1 = f1.GetParameter("p1")
#print("Steigung Background ohne Cut: ", p1)
#print("Achsenabschnitt Background ohne Cut: ", p0)
#print(" ")
#print(" ")
f3 = ROOT.TF1("f3", "{p0}+x*{p1}".format(p0=p0, p1=p1), 0, xmax) #ursprünglicher Background
f3.SetLineColor(1) #ursprünglicher Background
f3.SetLineStyle(4) #ursprünglicher Background
f3.Draw("same")
%jsroot
c1.Draw()
h2.GetXaxis().SetTitle("m(p K^{-} #pi^{+}) [MeV]")
h2.GetYaxis().SetTitle("Eintr#ddot{a}ge pro 1.1MeV [1/1.1MeV]")
c1.SetLeftMargin(0.12)
leg=ROOT.TLegend(0.8,0.7,0.99,0.99)
leg.AddEntry(h2,"Ungefilterte Daten","l")
leg.AddEntry(h1,"Gefilterte Daten","l")
leg.AddEntry(f22,"Signal","l")
leg.Draw()

Signal: 78386.3416429361
Signal + Untergrund: 196946.34158218425
Reinheit: 0.6308792436592233
Breite: 6.595954612372771
 


Warning in <TFile::Append>: Replacing existing TH1: h1 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h2 (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: myCanvasName


### Plots Omega_c
#### Anpassen Cuts mit Slider
In der nächsten Zelle sollst Du zum ersten Mal selsbt arbeiten, hier beginnt die eigentliche Datenanalyse! Es kann zunächst einen kurzen Moment dauern, bis die Histogramme erstellt sind, aber dann kann es direkt losgehen. Mithilfe der Schieberegler können die einzelnen Cuts angepasst und so die Daten zugeschnitten werden. Eine Beschreibung, was welcher Cut bedeutet, findest du auf dem Aufgabenblatt.

Am besten wartest Du immer kurz (5-10s) bis der zuletzt angewendete Cut von den Histogrammen übernommen wurde.

In [9]:
%matplotlib widget

df = df.assign(Xic= df["lab0_M"] * 0 + 2467.89)
gf = gf.assign(Xic= gf["lab0_M"] * 0 + 2467.89)


fig = plt.figure()
fig.set_size_inches(10, 6)
fig.canvas.header_visible = False

ax = plt.subplot2grid((13,14),(1,1),colspan = 5, rowspan = 5)
bx = plt.subplot2grid((13,14),(7,1),colspan = 5, rowspan = 5)
cx = plt.subplot2grid((13,14),(1,7),colspan = 3, rowspan = 3)
ex = plt.subplot2grid((13,14),(5,7),colspan = 3, rowspan = 3)
gx = plt.subplot2grid((13,14),(9,7),colspan = 3, rowspan = 3)
dx = plt.subplot2grid((13,14),(1,11),colspan = 3, rowspan = 3)
fx = plt.subplot2grid((13,14),(5,11),colspan = 3, rowspan = 3)
hx = plt.subplot2grid((13,14),(9,11),colspan = 3, rowspan = 3)


plt.tight_layout()

print("Bedingung 2: Wahrscheinlichkeit, dass ein Teilchen ein Proton ist")
print("Bedingung 3: Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt")
print("Bedingung 4: Qualität der Flugdistanz bezüglich des Kollisionspunkts")
print("Bedingung 5: Qualität des Fits mit erwarteten Massenhypothesen (= DTF)")
print("Bedingung 6: Wahrscheinlichkeit, dass ein Teilchen ein Kaon ist")
print("Bedingung 7: Größe des Transversalimpulses")
print("Eine kurze Erinnerung: Ist der Slider auf der linken Seite, wird wenig eingeschränkt, rechts schränkt er stark ein.")

@widgets.interact(Cut2=widgets.FloatSlider(min = 0, max = 1, step = 0.05, description = "Bed.2 $P(p)$",continuous_update = False), Cut3=widgets.FloatSlider(value=0,min = 0, max = 11, step = 0.1, description = "Bed.3 $\chi^2_{Stoß}$",continuous_update = False), Cut4=widgets.IntSlider(min = 0, max = 500, step = 2, description = "Bed.4 $\chi^2_{Flug}$",continuous_update = False),Cut5=widgets.FloatSlider(min = 0, max = 30, step = 0.1, description = "Bed.5 $\chi^2_{DTF}$",continuous_update = False), Cut6=widgets.FloatSlider(value=0,min = 0, max = 1, step = 0.01, description = "Bed.6 $P(K^-)$",continuous_update = False), Cut7=widgets.IntSlider(min = 0, max = 10000, step = 20, description = "Bed.7 $p_T$",continuous_update = False))
def update(Cut2=0, Cut3=0, Cut4=0, Cut5=0, Cut6=0, Cut7=0):
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    fx.cla()
    gx.cla()
    hx.cla()

    
    ax.set_title("Kombination $\Xi_c^+$ mit $K^-$")
    ax.set_ylabel("Einträge pro 2.2MeV [$1/2.2MeV$]")
    ax.set_xlabel("$m(\Xi_c^+ K^-)$ [$MeV$]")
    
    bx.set_title("Kombination $\Xi_c^+$ mit $K^+$")
    bx.set_ylabel("Einträge pro 2.2MeV [$1/2.2MeV$]")
    bx.set_xlabel("$m(\Xi_c^+ K^+)$ [$MeV$]")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Bedingung 2: Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("Bedingung 3: Stoßparameter")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("Bedingung 4: Flugdistanz")

    fx.set_title("")
    fx.set_ylabel("Einträge")
    fx.set_xlabel("Bedingung 5: Qualität DTF####")

    gx.set_title("")
    gx.set_ylabel("Einträge")
    gx.set_xlabel("Bedingung 6: Kaon")
    
    hx.set_title("")
    hx.set_ylabel("Einträge")
    hx.set_xlabel("Bedingung 7: $p_T$")
        
    ax.hist(df["lab0_M"]-df["lab1_M"]+df["Xic"][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)][df["lab1_FDCHI2_OWNPV"]>Cut4][df["lab0_CHI2NDOF_DTF_Xic"]<(30-Cut5)][df["lab3_ProbNNk"]>Cut6][df["lab0_PT"]>Cut7], bins=100, range =(2980,3150), color = "green")
    bx.hist(gf["lab0_M"]-gf["lab1_M"]+gf["Xic"][gf["lab2_ProbNNp"]>Cut2][gf["lab1_IPCHI2_OWNPV"]<(11-Cut3)][gf["lab1_FDCHI2_OWNPV"]>Cut4][gf["lab0_CHI2NDOF_DTF_Xic"]<(30-Cut5)][gf["lab3_ProbNNk"]>Cut6][gf["lab0_PT"]>Cut7], bins=100, range =(2980,3150), color = "darkgreen")
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))
    fx.hist(df["lab0_CHI2NDOF_DTF_Xic"][df["lab0_CHI2NDOF_DTF_Xic"]<(30-Cut5)], bins=100, range =(0,30))
    gx.hist(df["lab3_ProbNNk"][df["lab3_ProbNNk"]>Cut6], bins=100, range =(0,1))
    hx.hist(df["lab0_PT"][df["lab0_PT"]>Cut7], bins=100, range =(0,20000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Bedingung 2: Wahrscheinlichkeit, dass ein Teilchen ein Proton ist
Bedingung 3: Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt
Bedingung 4: Qualität der Flugdistanz bezüglich des Kollisionspunkts
Bedingung 5: Qualität des Fits mit erwarteten Massenhypothesen (= DTF)
Bedingung 6: Wahrscheinlichkeit, dass ein Teilchen ein Kaon ist
Bedingung 7: Größe des Transversalimpulses
Eine kurze Erinnerung: Ist der Slider auf der linken Seite, wird wenig eingeschränkt, rechts schränkt er stark ein.


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Bed.2 $P(p)$', max=1.0, ste…

### Plot for Xi_c
#### Read value in ROOT
Die nächste Zelle fordert Dich auf, die Werte für die Cuts anzugeben, die Du in dem vorherigen Teil mit den Schiebereglern erörtert hast.

Als nächstes wird dann ein anderes Histogramm erzeugt, was nur das Spektrum von Xi_c zeigt. In diesem kannst Du Dir mit dem Mauszeiger Werte anzeigen lassen, indem Du den Mauszeiger an die Stelle in das Histogramm bewegst, von der Du den Wert wissen möchtest. Zusätzlich wird dir ein Bereich an der X-Achse angezeigt, sodass Du besser ablesen kannst.

In [11]:
ROOT.gStyle.SetOptStat(0) #Entfernen HistLegende

#Hier die Cuts aus der ersten Teilaufgabe übernehmen
cut12 = ROOT.TCut("lab2_ProbNNp>0.4")
cut13 = ROOT.TCut("log(lab1_IPCHI2_OWNPV)<3")
cut14 = ROOT.TCut("lab1_FDCHI2_OWNPV>100")
cut10 = ROOT.TCut(cut12+cut13+cut14)

#Hier die Cuts aus der zweiten Teilaufgabe übernehmen
cut21 = ROOT.TCut("lab0_CHI2NDOF_DTF_Xic<2")
cut22 = ROOT.TCut("lab3_ProbNNk>0.7")
cut23 = ROOT.TCut("lab0_PT>4500")
cut20 = ROOT.TCut(cut21+cut22+cut23)

#Erstellen Hist, Kombinieren Hist
h2 = ROOT.TH1F("h2","Kombination #Xi_{c}^{+} mit K^{#pm}",100,2980,3200)
h3 = ROOT.TH1F("h3","h3",100,2980,3200)
c2 = ROOT.TCanvas("myCanvasName2","Titel2",800,600)
tree1.Draw("lab0_M-lab1_M+2467.89>>h2",cut10+cut20)
tree2.Draw("lab0_M-lab1_M+2467.89>>h3",cut10+cut20)
h2.SetLineWidth(2)
h2.Draw("MIN0")
h3.SetLineColor(2)
h3.SetLineWidth(2)
h3.Draw("same")
%jsroot
c2.Draw()
h2.GetXaxis().SetTitle("m(#Xi_{c}^{+} K^{#pm}) [MeV]")
h2.GetYaxis().SetTitle("Eintr#ddot{a}ge pro 2.2MeV")

leg=ROOT.TLegend(0.85,0.8,0.99,0.99)
leg.AddEntry(h2,"#Xi_{c}^{+} K^{-}","l")
leg.AddEntry(h3,"#Xi_{c}^{+} K^{+}","l")
leg.Draw()

Warning in <TFile::Append>: Replacing existing TH1: h2 (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h3 (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: myCanvasName2


## Zusatz
### Plots $\Omega_c$ WS
#### Anpassen Cuts mit Slider
In der nächsten Zelle sollst Du zum ersten Mal selsbt arbeiten, hier beginnt die eigentliche Datenanalyse! Es kann zunächst einen kurzen Moment dauern, bis die Histogramme erstellt sind, aber dann kann es direkt losgehen. Mithilfe der Schieberegler können die einzelnen Cuts angepasst und so die Daten zugeschnitten werden. Eine Beschreibung, was welcher Cut bedeutet, findest du auf dem Aufgabenblatt.

Am besten wartest Du immer kurz (5-10s) bis der zuletzt angewendete Cut von den Histogrammen übernommen wurde.

In [14]:
%matplotlib widget

gf = gf.assign(Xic= gf["lab0_M"] * 0 + 2467.89)

fig, ((ax, bx,cx),(dx,ex,fx),(gx,hx,ix)) = plt.subplots(3,3)
fig.set_size_inches(16, 10)

@widgets.interact(Cut2=(0,0.97, 0.05), Cut3=(0.1,11,0.1), Cut4=(0,1000,5), Cut5=(0,400000,500), Cut6=(0,30,0.25), Cut7=(0,1,0.05), Cut8=(0,20000,100))
def update(Cut2=0, Cut3=11, Cut4=0, Cut5=0, Cut6=30, Cut7=0, Cut8=0):
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    fx.cla()
    gx.cla()
    hx.cla()
    ix.cla()
    
    ax.set_title("Masse Omega_c")
    ax.set_ylabel("Einträge")
    ax.set_xlabel("Masse in MeV")
    
    bx.set_title("Masse Xi_c")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Masse in MeV")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Wahrscheinlichkeit für Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("impact parameter chi2 of Xic candidate with respect to the PV")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("Flightdistance own PV lab1")
    
    fx.set_title("")
    fx.set_ylabel("Einträge")
    fx.set_xlabel("Impuls lab1")
    
    gx.set_title("")
    gx.set_ylabel("Einträge")
    gx.set_xlabel("lab0_CHI2NDOF_DTF_Xic")

    hx.set_title("")
    hx.set_ylabel("Einträge")
    hx.set_xlabel("Wahrscheinlichkeit für Kaon")
    
    ix.set_title("")
    ix.set_ylabel("Einträge")
    ix.set_xlabel("Transversalimpuls für lab0")
        
    ax.hist(gf["lab0_M"]-gf["lab1_M"]+gf["Xic"][gf["lab2_ProbNNp"]>Cut2][gf["lab1_IPCHI2_OWNPV"]<Cut3][gf["lab1_FDCHI2_OWNPV"]>Cut4][gf["lab1_P"]>Cut5][gf["lab0_CHI2NDOF_DTF_Xic"]<Cut6][gf["lab3_ProbNNk"]>Cut7][gf["lab0_PT"]>Cut8], bins=100, range =(2980,3150), color = "green")
    bx.hist(gf["lab1_M"][gf["lab2_ProbNNp"]>Cut2][gf["lab1_IPCHI2_OWNPV"]<Cut3][gf["lab1_FDCHI2_OWNPV"]>Cut4][gf["lab1_P"]>Cut5][gf["lab0_CHI2NDOF_DTF_Xic"]<Cut6][gf["lab3_ProbNNk"]>Cut7][gf["lab0_PT"]>Cut8], bins=100, range =(2420,2510), color = "darkorange")
    cx.hist(gf["lab2_ProbNNp"][gf["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(gf["lab1_IPCHI2_OWNPV"][gf["lab1_IPCHI2_OWNPV"]<Cut3], bins=100, range =(0,11))
    ex.hist(gf["lab1_FDCHI2_OWNPV"][gf["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))
    fx.hist(gf["lab1_P"][gf["lab1_P"]>Cut5], bins=100, range =(0,300000))
    gx.hist(gf["lab0_CHI2NDOF_DTF_Xic"][gf["lab0_CHI2NDOF_DTF_Xic"]<Cut6], bins=100, range =(0,30))
    hx.hist(gf["lab3_ProbNNk"][gf["lab3_ProbNNk"]>Cut7], bins=100, range =(0,1))
    ix.hist(gf["lab0_PT"][gf["lab0_PT"]>Cut8], bins=100, range =(0,20000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='Cut2', max=0.97, step=0.05), FloatSlider(value=11.0,…

# Alte Zellen

In [ ]:
#old Root DF

df = ROOT.RDataFrame("DecayTree","/eos/user/s/sharst/SWAN_projects/MasterclassTest/Data2_2015MU.root")
#df = ROOT.RDataFrame("DecayTree","/eos/home/sharst/SWAN_projects/MasterclassTest/Daten.root")

In [ ]:
#old Xic Plots


%matplotlib widget

fig, ((ax,bx,cx),(dx,ex,fx)) = plt.subplots(2,3)
fig.set_size_inches(15, 6)

print ("Cut 1: Wahrscheinlichkeit, dass ein detektiertes Teilchen ein Proton ist")
print ("Cut 2: **EINFÜGEN**")
print ("Cut 3: **EINFÜGEN**")
@widgets.interact(Cut1=(0,0.95, 0.05), Cut2=(0.1,3,0.1), Cut3=(0,500,10))
def update(Cut1=0, Cut2=3, Cut3=0):
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    fx.cla()
    
    ax.set_title("Masse Xi_c")
    ax.set_ylabel("Einträge")
    ax.set_xlabel("Masse in MeV")
    
    bx.set_title("Cut1")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Wahrscheinlichkeit für Proton")
    
    cx.set_title("Cut2")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("****")
    cx.set_yscale("log")
    
    dx.set_title("Cut3")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("***")
    
    ex.set_title("Cut4")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("***")
    
    fx.set_title("Cut5")
    fx.set_ylabel("Einträge")
    fx.set_xlabel("***")
    
    npa = df.Filter("lab2_ProbNNp>{cut1}".format(cut1=Cut1)).Filter("log(lab1_IPCHI2_OWNPV)<{cut2}".format(cut2=Cut2)).Filter("lab1_FDCHI2_OWNPV>{cut3}".format(cut3=Cut3)).AsNumpy()
    ax.hist(npa["lab1_M"], bins = 100, range = (2420,2510))
    bx.hist(npa["lab2_ProbNNp"], bins = 100, range = (0,1))
    cx.hist(npa["lab1_IPCHI2_OWNPV"], bins = 100, range = (0,20))
    dx.hist(npa["lab1_FDCHI2_OWNPV"], bins = 100, range = (0,450))
    ex.hist(npa["lab2_ProbNNe"], bins = 100, range = (0,1))
    fx.hist(npa["lab1_P"], bins = 100, range = (17000,350000))

In [24]:
#old Xic Plot Root

f = ROOT.TFile.Open("/eos/user/s/sharst/SWAN_projects/MasterclassTest/Daten.root")
tree = f.Get("DecayTree")

xmin = 2420
xmax = 2510

cu1 = input("Wert für Cut1 angeben: ")
cu2 = input("Wert für Cut2 angeben: ")
cu3 = input("Wert für Cut3 angeben: ")


cut11 = ROOT.TCut("lab2_ProbNNp>{}".format(cu1))
cut12 = ROOT.TCut("log(lab1_IPCHI2_OWNPV)<{}".format(cu2))
cut13 = ROOT.TCut("lab1_FDCHI2_OWNPV>{}".format(cu3))
cut10 = ROOT.TCut(cut11+cut12+cut13)


h1 = ROOT.TH1F("h1", "Xi", 100, xmin, xmax)
tree.Draw("lab1_M>>h1", cut10)
c1 = ROOT.TCanvas("myCanvasName","Titel",800,600)
h1.Draw()
%jsroot
c1.Draw()
h1.GetXaxis().SetTitle("X axis title")
h1.GetYaxis().SetTitle("Y axis title")

Wert für Cut1 angeben: .4
Wert für Cut2 angeben: 2.8
Wert für Cut3 angeben: 140


Warning in <TCanvas::Constructor>: Deleting canvas with same name: myCanvasName


In [6]:
#old Omegac Plots matplotlib

%matplotlib widget

fig, ax = plt.subplots()
ax.grid(True)

print("Cut 1: Wahrscheinlichkeit, dass ein detektiertes Teilchen ein Proton ist")
print("Cut 2: **Einfügen**")
print("Cut 3: **Einfügen**")
print("Cut 4: **Einfügen**")
print("Cut 5: Wahrscheinlichkeit, dass ein detektiertes Teilchen ein Kaon ist")
print("Cut 6: **Einfügen**")

@widgets.interact(Cut1=(0,1,0.05),Cut2=(0,5,0.1),Cut3=(0,500,10),Cut4=(0,5,0.1),Cut5=(0,1,0.05),Cut6=(0,10000,100))
def update(Cut1=0,Cut2=5,Cut3=0,Cut4=5,Cut5=0,Cut6=10000):
    ax.cla()
    npa = df.Filter("lab2_ProbNNp>{cut1}".format(cut1=Cut1)).Filter("log(lab1_IPCHI2_OWNPV)<{cut2}".format(cut2=Cut2)).Filter("lab1_FDCHI2_OWNPV>{cut3}".format(cut3=Cut3)).Filter("lab0_CHI2NDOF_DTF_Xic<{cut4}".format(cut4=Cut4)).Filter("lab3_ProbNNk>{cut5}".format(cut5=Cut5)).Filter("lab0_PT>{cut6}".format(cut6=Cut6)).AsNumpy(columns=["lab0_M","lab1_M"])
    ax.hist(npa["lab0_M"]-npa["lab1_M"]+2467, bins = 100, range =(2980,3150))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Cut 1: Wahrscheinlichkeit, dass ein detektiertes Teilchen ein Proton ist
Cut 2: **Einfügen**
Cut 3: **Einfügen**
Cut 4: **Einfügen**
Cut 5: **Einfügen**
Cut 6: **Einfügen**


interactive(children=(FloatSlider(value=0.0, description='Cut1', max=1.0, step=0.05), FloatSlider(value=5.0, d…

In [11]:
#old Omegac Plot Root

cu4 = input("Wert für Cut4 angeben: ")
cu5 = input("Wert für Cut5 angeben: ")
cu6 = input("Wert für Cut6 angeben: ")


cut21 = ROOT.TCut("lab0_CHI2NDOF_DTF_Xic<{}".format(cu4))
cut22 = ROOT.TCut("lab3_ProbNNk>{}".format(cu5))
cut23 = ROOT.TCut("lab0_PT>{}".format(cu6))
cut20 = ROOT.TCut(cut21+cut22+cut23)

h2 = ROOT.TH1F("h2","Omega",100,2980,3150)
c2 = ROOT.TCanvas("myCanvasName2","Titel2",800,600)
tree.Draw("lab0_M-lab1_M+2467.89>>h2",cut10+cut20)
h2.Draw("")
%jsroot
c2.Draw()
h2.GetXaxis().SetTitle("X axis title")
h2.GetYaxis().SetTitle("Y axis title")

Wert für Cut4 angeben: 2
Wert für Cut5 angeben: .7
Wert für Cut6 angeben: 4500


Warning in <TCanvas::Constructor>: Deleting canvas with same name: myCanvasName2


In [40]:
%matplotlib widget

fig, ax = plt.subplots()

df = df.assign(Xic= df["lab0_M"] * 0 + 2467)

@widgets.interact(Cut1=(0,0.97, 0.05), Cut2=(0.25,11,0.25), Cut3=(0,500,10), Cut4=(0,1,0.05), Cut5=(0,400000,1000))
def update(Cut1=0, Cut2=11, Cut3=0, Cut4=0, Cut5=0):
    ax.cla()
        
    ax.hist(df["lab0_M"]-df["lab1_M"]+df["Xic"][df["lab2_ProbNNp"]>Cut1][df["lab1_IPCHI2_OWNPV"]<Cut2][df["lab1_FDCHI2_OWNPV"]>Cut3][df["lab2_ProbNNe"]>Cut4][df["lab1_P"]>Cut5], bins=100, range =(2900,3500))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, description='Cut1', max=0.97, step=0.05), FloatSlider(value=11.0,…

,lab0_M,lab1_M,lab2_ProbNNp,lab1_IPCHI2_OWNPV,lab1_FDCHI2_OWNPV,lab0_CHI2NDOF_DTF_Xic,lab3_ProbNNk,lab0_PT,lab1_P,lab2_ProbNNe,Xic
0,2986.598634,2489.048398,0.370264,1.564308,324.050236,2.601928,0.976817,5231.324358,140889.297418,9.039508e-06,2467.0
1,3459.652785,2493.886236,0.451091,4.874641,328.347398,3.002620,0.301009,5818.262669,91955.241282,3.343199e-04,2467.0
2,3257.589138,2482.472809,0.861226,1.125599,144.068836,9.820177,0.991262,5367.397287,31151.952951,1.992916e-07,2467.0
3,3465.332939,2466.705115,0.481511,8.544077,41.432774,1.947516,0.485345,5345.653207,77779.209294,2.363602e-02,2467.0
4,3369.337698,2466.705115,0.481511,8.544077,41.432774,2.010060,0.485345,5376.485972,77779.209294,2.363602e-02,2467.0
...,...,...,...,...,...,...,...,...,...,...,...
1068772,2956.150106,2431.261712,0.469457,1.174319,275.082043,6.223449,0.975301,3921.125727,87620.765603,5.301443e-05,2467.0
1068773,3387.207621,2431.261712,0.469457,1.174319,275.082043,5.625250,0.975301,3700.094527,87620.765603,5.301443e-05,2467.0
1068774,3107.727160,2456.281292,0.501375,6.220686,268.320422,2.214079,0.880667,4262.244574,44240.203561,2.566573e-06,2467.0
1068775,3384.439917,2469.294647,0.918932,3.057070,89.176439,0.791783,0.998145,6691.661145,66256.820576,4.087395e-05,2467.0


In [16]:
fig = plt.figure()
  
axes1 = plt.subplot2grid((4, 4), (0, 0),
                         colspan = 4)
  
axes2 = plt.subplot2grid((4, 4), (1, 0),
                         colspan = 3)
  
axes3 = plt.subplot2grid((4, 4), (1, 2), 
                         rowspan = 3)
  
axes4 = plt.subplot2grid((4, 4), (2, 0))
axes5 = plt.subplot2grid((4, 4), (2, 1))
  
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig = plt.figure()

a1 = plt.subplot2grid((4,4),(2,3),colspan = 2)
a2 = plt.subplot2grid((4,4),(0,3), rowspan = 2, colspan = 1)
a3 = plt.subplot2grid((4,4),(0,0),rowspan = 4, colspan = 3)

  
fig.tight_layout()

x = np.arange(1,10)
a2.plot(x, x*x)
a2.set_title('square')
a1.plot(x, np.exp(x))
a1.set_title('exp')
a3.plot(x, np.log(x))
a3.set_title('log')
plt.tight_layout()
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [64]:
fig = plt.figure()

ax = plt.subplot2grid((6,10),(0,0),colspan = 4, rowspan = 5)
bx = plt.subplot2grid((6,10),(0,4),colspan = 3, rowspan = 2)
cx = plt.subplot2grid((6,10),(3,4),colspan = 3, rowspan = 2)
dx = plt.subplot2grid((6,10),(0,7),colspan = 3, rowspan = 2)
ex = plt.subplot2grid((6,10),(3,7),colspan = 3, rowspan = 2)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [91]:
def gauss_lbg(x, par): #we assume that bin width is 1 by default
    # par[0]	background constant
    # par[1]	background slope
    # par[2]	gauss width
    # par[3]	gauss constant
    # par[4]	gauss mean
    sqrt2pi , sqrt2 = np.sqrt(2*np.pi) , np.sqrt(2)
    if par[2] == 0: par[2]=1
    arg = (x[0] - par[4])/(sqrt2*par[2])
    return par[0] + x[0]*par[1] + 1/(sqrt2pi*par[2]) * par[3] * np.exp(-arg*arg)



gauss_linbg_fit = ROOT.TF1('gauss_linbg_fit','[0] + x*[1] + 1/(TMath::Sqrt(2*TMath::Pi())*[2]) * [3] * TMath::Exp(-((x - [4])/(TMath::Sqrt(2.)*[2]))*((x - [4])/(TMath::Sqrt(2.)*[2])))', 2455, 2480)

In [ ]:
xmin = 2420
xmax = 2510

cu1 = input("Wert für Cut1 angeben: ")
cu2 = input("Wert für Cut2 angeben: ")
cu3 = input("Wert für Cut3 angeben: ")
cu4 = input("Wert für Cut4 angeben: ")
cu5 = input("Wert für Cut5 angeben: ")

cut11 = ROOT.TCut("lab2_ProbNNe>{}".format(cu1))
cut12 = ROOT.TCut("lab2_ProbNNp>{}".format(cu2))
cut13 = ROOT.TCut("log(lab1_IPCHI2_OWNPV)<{}".format(cu3))
cut14 = ROOT.TCut("lab1_FDCHI2_OWNPV>{}".format(cu4))
cut15 = ROOT.TCut("lab1_P>{}".format(cu5))
cut10 = ROOT.TCut(cut11+cut12+cut13+cut14+cut15)

f1 = ROOT.TF1("f1", [0] * x +[1])


h1 = ROOT.TH1F("h1", "#Xi_{c}^{+}", 100, xmin, xmax)
tree.Draw("lab1_M>>h1", cut10)
c1 = ROOT.TCanvas("myCanvasName","Titel",800,600)
h1.Draw()
h1.Fit("f1")
%jsroot
c1.Draw()
h1.GetXaxis().SetTitle("m(p K^{-} #pi^{+}) [MeV]")
h1.GetYaxis().SetTitle("Eintr#ddot{a}ge pro MeV [1/MeV]")

In [73]:
class myGaus:
    def __call__( self, t, par ):
        a = par[0] #amplitude
        c = par[1] #center
        s = par[2] #sigma
        x = t[0]   
        tmp = -1.*(x-c)*(x-c)/2. /(s*s)
        return a * math.exp( tmp )

In [36]:
fig = plt.figure()
fig.set_size_inches(10, 6)
fig.canvas.header_visible = False

ax = plt.subplot2grid((12,16),(0,1),colspan = 5, rowspan = 5)
bx = plt.subplot2grid((12,16),(6,1),colspan = 5, rowspan = 5)
cx = plt.subplot2grid((12,16),(0,7),colspan = 3, rowspan = 3)
dx = plt.subplot2grid((12,16),(4,7),colspan = 3, rowspan = 3)
ex = plt.subplot2grid((12,16),(8,7),colspan = 3, rowspan = 3)
fx = plt.subplot2grid((12,16),(0,11),colspan = 3, rowspan = 3)
gx = plt.subplot2grid((12,16),(4,11),colspan = 3, rowspan = 3)
hx = plt.subplot2grid((12,16),(8,11),colspan = 3, rowspan = 3)


plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
%matplotlib widget

df = df.assign(Xic= df["lab0_M"] * 0 + 2467.89)
gf = gf.assign(Xic= gf["lab0_M"] * 0 + 2467.89)


fig = plt.figure()
fig.set_size_inches(10, 6)
fig.canvas.header_visible = False

ax = plt.subplot2grid((12,16),(0,1),colspan = 5, rowspan = 5)
bx = plt.subplot2grid((12,16),(6,1),colspan = 5, rowspan = 5)
cx = plt.subplot2grid((12,16),(0,7),colspan = 3, rowspan = 3)
dx = plt.subplot2grid((12,16),(4,7),colspan = 3, rowspan = 3)
ex = plt.subplot2grid((12,16),(8,7),colspan = 3, rowspan = 3)
fx = plt.subplot2grid((12,16),(0,11),colspan = 3, rowspan = 3)
gx = plt.subplot2grid((12,16),(4,11),colspan = 3, rowspan = 3)
hx = plt.subplot2grid((12,16),(8,11),colspan = 3, rowspan = 3)


plt.tight_layout()

@widgets.interact(Cut2=(0,0.97, 0.05), Cut3=(0.1,11,0.1), Cut4=(0,1000,5), Cut6=(0,30,0.25), Cut7=(0,1,0.05), Cut8=(0,20000,100))
def update(Cut2=0, Cut3=11, Cut4=0, Cut6=30, Cut7=0, Cut8=0):
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    fx.cla()
    gx.cla()
    hx.cla()
    ix.cla()
    
    ax.set_title("Masse Omega_c")
    ax.set_ylabel("Einträge")
    ax.set_xlabel("Masse in MeV")
    
    bx.set_title("Masse Xi_c")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Masse in MeV")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Wahrscheinlichkeit für Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("impact parameter chi2 of Xic candidate with respect to the PV")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("Flightdistance own PV lab1")
    
    fx.set_title("")
    fx.set_ylabel("Einträge")
    fx.set_xlabel("Impuls lab1")
    
    gx.set_title("")
    gx.set_ylabel("Einträge")
    gx.set_xlabel("lab0_CHI2NDOF_DTF_Xic")

    hx.set_title("")
    hx.set_ylabel("Einträge")
    hx.set_xlabel("Wahrscheinlichkeit für Kaon")
    
    ix.set_title("")
    ix.set_ylabel("Einträge")
    ix.set_xlabel("Transversalimpuls für lab0")
        
    ax.hist(df["lab0_M"]-df["lab1_M"]+df["Xic"][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<Cut3][df["lab1_FDCHI2_OWNPV"]>Cut4][df["lab0_CHI2NDOF_DTF_Xic"]<Cut6][df["lab3_ProbNNk"]>Cut7][df["lab0_PT"]>Cut8], bins=100, range =(2980,3150), color = "green")
    bx.hist(gf["lab0_M"]-gf["lab1_M"]+gf["Xic"][gf["lab2_ProbNNp"]>Cut2][gf["lab1_IPCHI2_OWNPV"]<Cut3][gf["lab1_FDCHI2_OWNPV"]>Cut4][gf["lab0_CHI2NDOF_DTF_Xic"]<Cut6][gf["lab3_ProbNNk"]>Cut7][gf["lab0_PT"]>Cut8], bins=100, range =(2980,3150), color = "darkgreen")
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<Cut3], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))
    gx.hist(df["lab0_CHI2NDOF_DTF_Xic"][df["lab0_CHI2NDOF_DTF_Xic"]<Cut6], bins=100, range =(0,30))
    hx.hist(df["lab3_ProbNNk"][df["lab3_ProbNNk"]>Cut7], bins=100, range =(0,1))
    ix.hist(df["lab0_PT"][df["lab0_PT"]>Cut8], bins=100, range =(0,20000))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
widgets.FloatSlider(
    value=0,
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
widgets.FloatSlider(
    value=0,
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
widgets.FloatSlider(
    value=0,
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
widgets.FloatSlider(
    value=0,
    min=0,
    max=10.0,
    step=0.1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

FloatSlider(value=0.0, continuous_update=False, description='Test:', max=10.0, readout_format='.1f')

In [90]:
%matplotlib widget

fig = plt.figure()
fig.set_size_inches(10, 5)
fig.canvas.header_visible = False

ax = plt.subplot2grid((6,16),(0,1),colspan = 7, rowspan = 5)
bx = plt.subplot2grid((6,16),(0,9),colspan = 3, rowspan = 2)
cx = plt.subplot2grid((6,16),(3,9),colspan = 3, rowspan = 2)
dx = plt.subplot2grid((6,16),(0,13),colspan = 3, rowspan = 2)
ex = plt.subplot2grid((6,16),(3,13),colspan = 3, rowspan = 2)

plt.tight_layout()



lb1 = widgets.Label("Cut1: Test")
display(lb1)
Cut1 = widgets.FloatSlider(value=0,min=0,max=1.0,step=0.01,disabled=False,continuous_update=False)
display(Cut1)
lb2 = widgets.Label("Cut2: Test")
display(lb2)
Cut2 = widgets.FloatSlider(value=0,min=0,max=1.0,step=0.01,disabled=False,continuous_update=False)
display(Cut2)
lb3 = widgets.Label("Cut3: Test")
display(lb3)
Cut3 = widgets.FloatSlider(value=30,min=0,max=30.0,step=0.5,disabled=False,continuous_update=False)
display(Cut3)
lb4 = widgets.Label("Cut4: Test")
display(lb4)
Cut4 = widgets.FloatSlider(value=0,min=0,max=500.0,step=2,disabled=False,continuous_update=False)
display(Cut4)
def on_value_change(Cut1):
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    
    ax.set_title("$\Xi_c^+$")
    ax.set_ylabel("Einträge pro MeV [$1/MeV$]")
    ax.set_xlabel("$m(pK^-\pi^+)$ [$MeV$]")
    
    bx.set_title("")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Wahrscheinlichkeit für Elektron")
    bx.set_yscale("log")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Wahrscheinlichkeit für Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("****")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("***")
    
    ax.hist(df["lab1_M"], bins=100, range =(2420,2510), color = "brown")
    ax.hist(df["lab1_M"][df["lab2_ProbNNe"]>Cut1][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<Cut3][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(2420,2510), color = "darkorange")
    bx.hist(df["lab2_ProbNNe"][df["lab2_ProbNNe"]>Cut1], bins=100, range =(0,1))
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<Cut3], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Label(value='Cut1: Test')

FloatSlider(value=0.0, continuous_update=False, max=1.0, step=0.01)

Label(value='Cut2: Test')

FloatSlider(value=0.0, continuous_update=False, max=1.0, step=0.01)

Label(value='Cut3: Test')

FloatSlider(value=30.0, continuous_update=False, max=30.0, step=0.5)

Label(value='Cut4: Test')

FloatSlider(value=0.0, continuous_update=False, max=500.0, step=2.0)

In [114]:
%matplotlib widget

fig = plt.figure()
fig.set_size_inches(10, 5)
fig.canvas.header_visible = False

ax = plt.subplot2grid((6,16),(0,1),colspan = 7, rowspan = 5)
bx = plt.subplot2grid((6,16),(0,9),colspan = 3, rowspan = 2)
cx = plt.subplot2grid((6,16),(3,9),colspan = 3, rowspan = 2)
dx = plt.subplot2grid((6,16),(0,13),colspan = 3, rowspan = 2)
ex = plt.subplot2grid((6,16),(3,13),colspan = 3, rowspan = 2)


plt.tight_layout()

#fig, ((ax,bx,cx),(dx,ex)) = plt.subplots(2,3)
#fig.set_size_inches(16, 7)

@widgets.interact(Text1 = widgets.Label("Test bla bla"),Text2=widgets.Label("asdfpsakjflöaj"),Cut1=FloatSlider(min = 0, max = 1, step = 0.05, description = "Bedingung 1:",continuous_update = False)) #, Cut2=(0,1,0.05), Cut3=(0.1,11,0.1), Cut4=(0,1000,5)
def update(Text1,Cut1=0,  Cut2=0, Cut3=11, Cut4=0): #, Cut3=11, Cut4=0
    
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    
    ax.set_title("$\Xi_c^+$")
    ax.set_ylabel("Einträge pro MeV [$1/MeV$]")
    ax.set_xlabel("$m(pK^-\pi^+)$ [$MeV$]")
    
    bx.set_title("")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Wahrscheinlichkeit für Elektron")
    bx.set_yscale("log")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Wahrscheinlichkeit für Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("****")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("***")
    
    ax.hist(df["lab1_M"], bins=100, range =(2420,2510), color = "brown")
    ax.hist(df["lab1_M"][df["lab2_ProbNNe"]>Cut1][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<Cut3][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(2420,2510), color = "darkorange")
    bx.hist(df["lab2_ProbNNe"][df["lab2_ProbNNe"]>Cut1], bins=100, range =(0,1))
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<Cut3], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Label(value='Test bla bla', description='Text1'), FloatSlider(value=0.0, continuous_upda…

In [108]:
%matplotlib widget

df = df.assign(Xic= df["lab0_M"] * 0 + 2467.89)
gf = gf.assign(Xic= gf["lab0_M"] * 0 + 2467.89)


fig = plt.figure()
fig.set_size_inches(10, 6)
fig.canvas.header_visible = False

ax = plt.subplot2grid((12,14),(0,1),colspan = 5, rowspan = 5)
bx = plt.subplot2grid((12,14),(6,1),colspan = 5, rowspan = 5)
cx = plt.subplot2grid((12,14),(0,7),colspan = 3, rowspan = 3)
dx = plt.subplot2grid((12,14),(4,7),colspan = 3, rowspan = 3)
ex = plt.subplot2grid((12,14),(8,7),colspan = 3, rowspan = 3)
fx = plt.subplot2grid((12,14),(0,11),colspan = 3, rowspan = 3)
gx = plt.subplot2grid((12,14),(4,11),colspan = 3, rowspan = 3)
hx = plt.subplot2grid((12,14),(8,11),colspan = 3, rowspan = 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [254]:
xmin = 2420
xmax = 2510
ROOT.gStyle.SetOptStat(0) #Entfernen HistLegende

#Erstellen der Cuts
cut11 = ROOT.TCut("lab2_ProbNNe>0") #Einfügen W'keit für Elektron
cut12 = ROOT.TCut("lab2_ProbNNp>0.45") #Einfügen W'keit für Proton
cut13 = ROOT.TCut("lab1_IPCHI2_OWNPV<3") #Einfügen IP Cut
cut14 = ROOT.TCut("lab1_FDCHI2_OWNPV>100") #Einfügen FD Cut
cut10 = ROOT.TCut(cut11+cut12+cut13+cut14) #Erstellen Gesamtcut

#Erstellen Fitfunktion: linBG + Gauss + Gauss
f1 = ROOT.TF1('f1','[0] + x*[1] + [3] * exp(-(x - [4])^2/(2*[2]*[2]))+ [5] * exp(-(x - [7])^2/(2*[6]*[6]))', xmin, xmax) #Fitfunktion
f1.SetParameter(4, 2468) #Parameter für Gauss Tendenz setzen Mean
f1.SetParameter(3,1) #Parameter für Gauss Tendenz setzen existenz
f1.SetParameter(5,1) #existenz Gauss2
f1.SetParameter(7, 2468) #Mean Gauss2

#Erstellen und Füllen der Histogramme
h1 = ROOT.TH1F("h1", "Kombination der Teilchen", 100, xmin, xmax) #Hist mit Cut
tree1.Draw("lab1_M>>h1",cut10) #Füllen Hist mit Cut
c6 = ROOT.TCanvas("myCanvasName","Titel",800,600) #Erstellen Canvas
h1.Draw("MIN0") #Hist mC zeichnen in gleiches TCanvas

#erste Fits, Fitparameter für Integrale nehmen
f1.SetLineColor(ROOT.kBlue-6)
h1.Fit("f1","RQ") #Hist mC fitten
p10 = f1.GetParameter("p0") #getParForBG#Y-Intersect
p11 = f1.GetParameter("p1") #GetParForBG#slope
p22 = f1.GetParameter("p2") #GetParForSignal #sigma
p23 = f1.GetParameter("p3") #Signal
p24 = f1.GetParameter("p4") #Signal #Mean
p25 = f1.GetParameter("p5") #Signal 
p26 = f1.GetParameter("p6") #Signal #sigma
p27 = f1.GetParameter("p7") #Signal #mean

#print("Steigung Background mit Cut: ", p11) #SlopeOfBGCUt
#print("Achsenabschnitt Background mit Cut: ", p10) #Y-IntersectOfBGCut
#print(" ") #Space

#Bestimmen der Funktione für Integrale, kosmetische Anpassungen Fits
f21 = ROOT.TF1("f21", "{p10}+x*{p11}".format(p10=p10, p11=p11), 0, xmax) #BGFunctionCut
f21.SetLineColor(1) #BGCut
f21.SetLineStyle(4) #BGCut
f21.Draw("same") #Draw BGCut
f22 = ROOT.TF1("f22", "{p23} * exp(-(x - {p24})^2/(2*{p22}*{p22}))+ {p25} * exp(-(x - {p27})^2/(2*{p26}*{p26}))".format(p22=p22,p23=p23,p24=p24,p25=p25,p26=p26,p27=p27), xmin,xmax) #only SignalPeak Fit
f23 = ROOT.TF1("f23", "{p10}+x*{p11} +{p23} * exp(-(x - {p24})^2/(2*{p22}*{p22}))+ {p25} * exp(-(x - {p27})^2/(2*{p26}*{p26}))".format(p10=p10,p11=p11,p22=p22,p23=p23,p24=p24,p25=p25,p26=p26,p27=p27), xmin,xmax)
f22.SetLineColor(ROOT.kBlue+2)
f22.SetLineWidth(3)
f22.Draw("same") #Draw SignalPeak
SB = f23.Integral(p24 - 2 * abs(p22), p24 + 2 * abs(p22)) #integrate Sign+Bg in signalregion (mean+-2sigma)
Signal = f22.Integral(p24 - 2*abs(p22), p24 + 2*abs(p22)) #integrate signal in signalregion ("")
f23.SetLineColor(1)
f23.SetLineStyle(4)
f23.Draw("same")
Bg = f21.Integral(xmin, xmax) #Integrate Background whole Dataset
print(Bg) #PrintBG

#Variablen für die
Varianz = abs(p22)
print("Signal:", Signal)
print("Signal + Background:", SB)
Purity = Signal/SB
print("Purity:", Purity)
print("Varianz:", Varianz)
print(" ")

f1.SetParameter(4, 2468) #Parameter für Gauss Tendenz setzen Mean
f1.SetParameter(3,1) #Parameter für Gauss Tendenz setzen existenz
f1.SetParameter(5,1) #existenz Gauss2
f1.SetParameter(7, 2468) #Mean Gauss2
f1.SetLineColor(ROOT.kRed)
#h2.Fit("f1","RQ") #ursprünglicher Signal
p0 = f1.GetParameter("p0")
p1 = f1.GetParameter("p1")
#f3 = ROOT.TF1("f3", "{p0}+x*{p1}".format(p0=p0, p1=p1), 0, xmax) #ursprünglicher Background
#f3.SetLineColor(1) #ursprünglicher Background
#f3.SetLineStyle(4) #ursprünglicher Background
#f3.Draw("same")
%jsroot
c6.Draw()
h1.GetXaxis().SetTitle("m(p K^{-} #pi^{+}) [MeV]")
h1.GetYaxis().SetTitle("Eintr#ddot{a}ge pro 1.1MeV [1/1.1MeV]")
c6.SetLeftMargin(0.12)
leg=ROOT.TLegend(0.8,0.7,0.99,0.99)
#leg.AddEntry(h2,"Ungefilterte Daten","l")
leg.AddEntry(h1,"Daten","l")
leg.AddEntry(f23,"Hintergrund","l")
leg.AddEntry(f22,"Signal","l")
leg.Draw()

407986.42498478317
Signal: 78386.3416429361
Signal + Background: 196946.34158218425
Purity: 0.3980086200800336
Varianz: 6.595954612372771
 


Warning in <TFile::Append>: Replacing existing TH1: h1 (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: myCanvasName


In [263]:
%matplotlib widget

fig = plt.figure()
fig.set_size_inches(10, 5)
fig.canvas.header_visible = False

ax = plt.subplot2grid((7,16),(1,1),colspan = 7, rowspan = 5)
bx = plt.subplot2grid((7,16),(1,9),colspan = 3, rowspan = 2)
dx = plt.subplot2grid((7,16),(4,9),colspan = 3, rowspan = 2)
cx = plt.subplot2grid((7,16),(1,13),colspan = 3, rowspan = 2)
ex = plt.subplot2grid((7,16),(4,13),colspan = 3, rowspan = 2)


plt.tight_layout()

a = 0

#fig, ((ax,bx,cx),(dx,ex)) = plt.subplots(2,3)
#fig.set_size_inches(16, 7)
#print("Filter 2: Wahrscheinlichkeit, dass ein Teilchen ein Proton ist")
#print("Filter 3: Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt")
#print("Filter 4: Qualität der Flugdistanz bezüglich des Kollisionspunkts")


@widgets.interact(Cut1=FloatSlider(min = 0, max = 1, step = 0.05, description = "Ftr.1 $P(e^-)$",continuous_update = False), Cut2=FloatSlider(min = 0, max = 1, step = 0.05, description = "Ftr.2 $P(p)$",continuous_update = False), Cut3=FloatSlider(value=0,min = 0, max = 11, step = 0.1, description = "Ftr.3 $\chi^2_{Stoß}$",continuous_update = False), Cut4=IntSlider(min = 0, max = 500, step = 2, description = "Ftr.4 $\chi^2_{Flug}$",continuous_update = False))
def update(Cut1, Cut2=0, Cut3=11, Cut4=0):
    
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    
    ax.set_title("Kombination $p$ mit $K^-$ und $\pi^+$")
    ax.set_ylabel("Einträge pro 1.1MeV")
    ax.set_xlabel("$m(pK^-\pi^+)$ [$MeV$]")
    
    bx.set_title("")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Filter 1: Elektron")
    bx.set_yscale("log")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Filter 2: Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("Filter 3: Stoßparameter")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("Filter 4: Flugdistanz")
    
    ax.hist(df["lab1_M"], bins=100, range =(2420,2510), color = "brown")
    ax.hist(df["lab1_M"][df["lab2_ProbNNe"]>Cut1][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(2420,2510), color = "darkorange")
    bx.hist(df["lab2_ProbNNe"][df["lab2_ProbNNe"]>Cut1], bins=100, range =(0,1))
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Ftr.1 $P(e^-)$', max=1.0, s…

In [17]:
a = math.sqrt(4)
print(a)

2.0
